In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
train_df = pd.read_csv("/kaggle/input/playground-series-s3e16/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s3e16/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s3e16/sample_submission.csv")

In [4]:
train_df = pd.get_dummies(train_df, columns=['Sex'])
test_df = pd.get_dummies(test_df, columns=['Sex'])

In [5]:
X_train = train_df.drop("Age", axis = 1)
y_train = train_df.Age

In [6]:
lgb_model = lgb.LGBMRegressor(
    categorical_feature= [0],
    task = 'predict',
    application = 'regression',
    objective = 'mae',
    boosting_type="gbdt",
    num_iterations = 2600,
    learning_rate = 0.09,
    num_leaves=9,
    tree_learner='feature',
    max_depth =12,
    min_data_in_leaf=10,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='mae',
    feature_fraction = 0.8,
    random_state=42)

In [7]:
lgb_model.fit(X_train,y_train)
predict = lgb_model.predict(test_df)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


In [8]:
submission

,id,Age
0,74051,10
1,74052,10
2,74053,10
3,74054,10
4,74055,10
...,...,...
49363,123414,10
49364,123415,10
49365,123416,10
49366,123417,10


In [9]:
print(len(predict))

49368


In [10]:
submission["Age"] = np.round(predict)
submission.to_csv('submission_lgb.csv', index=False)
submission_lgb = pd.read_csv('submission_lgb.csv')
submission_lgb

,id,Age
0,74051,7.0
1,74052,8.0
2,74053,10.0
3,74054,9.0
4,74055,8.0
...,...,...
49363,123414,9.0
49364,123415,8.0
49365,123416,13.0
49366,123417,9.0


In [11]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

cat_model1 = CatBoostRegressor(verbose=False, objective='MAE')

scores = -cross_val_score(cat_model1, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE:", np.mean(scores))

Cross-Validation MAE: 1.3645152658890103


In [12]:
cat_model1.fit(X_train,y_train)
predict_cat1 = cat_model1.predict(test_df)

In [13]:
submission["Age"] = np.round(predict_cat1)
submission.to_csv('submission_cat1.csv', index=False)
submission_cat1 = pd.read_csv('submission_cat1.csv')
submission_cat1

,id,Age
0,74051,7.0
1,74052,8.0
2,74053,10.0
3,74054,9.0
4,74055,8.0
...,...,...
49363,123414,9.0
49364,123415,8.0
49365,123416,13.0
49366,123417,9.0


In [14]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the CatBoostRegressor model
cat_model = CatBoostRegressor(verbose=False, objective='MAE')

# Define the hyperparameter grid
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300]
}

# Perform hyperparameter tuning using cross-validation
random_search = RandomizedSearchCV(
    cat_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=2,
    random_state=42
)
random_search.fit(X_train, y_train)

# Get the best model and its MAE score
best_model = random_search.best_estimator_
scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("CatBoost Cross-Validation MAE:", np.mean(scores))

# Make predictions
predict_cat = best_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_cat)

# Save predictions to a CSV file
submission.to_csv('submission_cat.csv', index=False)

# Read the submission file
submission_cat = pd.read_csv('submission_cat.csv')
print(submission_cat)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END .........depth=4, iterations=300, learning_rate=0.1; total time=   2.6s
[CV] END .........depth=4, iterations=300, learning_rate=0.1; total time=   2.6s
[CV] END .........depth=4, iterations=300, learning_rate=0.1; total time=   2.6s
[CV] END .........depth=4, iterations=300, learning_rate=0.1; total time=   2.6s
[CV] END .........depth=4, iterations=300, learning_rate=0.1; total time=   2.6s
[CV] END ........depth=6, iterations=200, learning_rate=0.05; total time=   2.1s
[CV] END ........depth=6, iterations=200, learning_rate=0.05; total time=   2.1s
[CV] END ........depth=6, iterations=200, learning_rate=0.05; total time=   2.1s
[CV] END ........depth=6, iterations=200, learning_rate=0.05; total time=   2.1s
[CV] END ........depth=6, iterations=200, learning_rate=0.05; total time=   2.1s
[CV] END ........depth=6, iterations=100, learning_rate=0.01; total time=   1.2s
[CV] END ........depth=6, iterations=100, learni

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the RandomForestRegressor model
rf_model = RandomForestRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 4, 8]
}

# Perform hyperparameter tuning using cross-validation
random_search = RandomizedSearchCV(
    rf_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=2,
    random_state=42
)
random_search.fit(X_train, y_train)

# Get the best model and its MAE score
best_model = random_search.best_estimator_
scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Random Forest Cross-Validation MAE:", np.mean(scores))

# Make predictions
predict_rf = best_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_rf)

# Save predictions to a CSV file
submission.to_csv('submission_rf.csv', index=False)

# Read the submission file
submission_rf = pd.read_csv('submission_rf.csv')
print(submission_rf)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=None, min_samples_split=8, n_estimators=300; total time= 1.8min
[CV] END max_depth=None, min_samples_split=8, n_estimators=300; total time= 1.9min
[CV] END max_depth=None, min_samples_split=8, n_estimators=300; total time= 1.9min
[CV] END max_depth=None, min_samples_split=8, n_estimators=300; total time= 1.9min
[CV] END max_depth=None, min_samples_split=8, n_estimators=300; total time= 1.9min
[CV] END .max_depth=5, min_samples_split=4, n_estimators=200; total time=  21.8s
[CV] END .max_depth=5, min_samples_split=4, n_estimators=200; total time=  21.9s
[CV] END .max_depth=5, min_samples_split=4, n_estimators=200; total time=  21.8s
[CV] END .max_depth=5, min_samples_split=4, n_estimators=200; total time=  21.7s
[CV] END .max_depth=5, min_samples_split=4, n_estimators=200; total time=  21.7s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=  10.9s
[CV] END .max_depth=5, min_samples_spl

In [16]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the XGBRegressor model
xgb_model = XGBRegressor()

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'n_estimators': [100, 200, 300]
}

# Perform hyperparameter tuning using cross-validation
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=2,
    random_state=42
)
random_search.fit(X_train, y_train)

# Get the best model and its MAE score
best_model = random_search.best_estimator_
scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("XGBoost Cross-Validation MAE:", np.mean(scores))

# Make predictions
predict_xgb = best_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_xgb)

# Save predictions to a CSV file
submission.to_csv('submission_xgb.csv', index=False)

# Read the submission file
submission_xgb = pd.read_csv('submission_xgb.csv')
print(submission_xgb)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time=  35.7s
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time=  36.0s
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time=  36.1s
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time=  35.8s
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time=  35.9s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  13.9s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  14.0s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  13.7s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  14.2s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  13.9s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=   3.5s
[CV] END ..learning_rate=0.05, max_depth=3, n_es

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

# Define the Linear Regression model
linear_model = LinearRegression()

# Train the model
linear_model.fit(X_train, y_train)

# Make predictions
predict_linear = linear_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_linear)

# Save predictions to a CSV file
submission.to_csv('submission_linear.csv', index=False)

# Read the submission file
submission_linear = pd.read_csv('submission_linear.csv')
print(submission_linear)


           id   Age
0       74051   8.0
1       74052   8.0
2       74053  10.0
3       74054  10.0
4       74055   8.0
...       ...   ...
49363  123414   9.0
49364  123415   8.0
49365  123416  12.0
49366  123417  10.0
49367  123418  13.0

[49368 rows x 2 columns]


In [18]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the GradientBoostingRegressor model
gb_model = GradientBoostingRegressor()

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'n_estimators': [100, 200, 300]
}

# Perform hyperparameter tuning using cross-validation
random_search = RandomizedSearchCV(
    gb_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=2,
    random_state=42
)
random_search.fit(X_train, y_train)

# Get the best model and its MAE score
best_model = random_search.best_estimator_
scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Gradient Boosting Cross-Validation MAE:", np.mean(scores))

# Make predictions
predict_gb = best_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_gb)

# Save predictions to a CSV file
submission.to_csv('submission_gb.csv', index=False)

# Read the submission file
submission_gb = pd.read_csv('submission_gb.csv')
print(submission_gb)



Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time= 1.4min
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time= 1.4min
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time= 1.4min
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time= 1.4min
[CV] END ..learning_rate=0.01, max_depth=9, n_estimators=300; total time= 1.4min
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  41.0s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  41.2s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  41.2s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  41.8s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  41.5s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=  10.6s
[CV] END ..learning_rate=0.05, max_depth=3, n_es

In [19]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Define the HistGradientBoostingRegressor model
hist_model = HistGradientBoostingRegressor()

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'max_iter': [100, 200, 300]
}

# Perform hyperparameter tuning using cross-validation
random_search = RandomizedSearchCV(
    hist_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=2,
    random_state=42
)
random_search.fit(X_train, y_train)

# Get the best model and its MAE score
best_model = random_search.best_estimator_
scores = -cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("HistGradientBoosting Cross-Validation MAE:", np.mean(scores))

# Make predictions
predict_hist = best_model.predict(test_df)

# Round the predictions
submission["Age"] = np.round(predict_hist)

# Save predictions to a CSV file
submission.to_csv('submission_hist.csv', index=False)

# Read the submission file
submission_hist = pd.read_csv('submission_hist.csv')
print(submission_hist)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/conda/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


[CV] END ......learning_rate=0.01, max_depth=9, max_iter=300; total time=   4.3s
[CV] END ......learning_rate=0.01, max_depth=9, max_iter=300; total time=   4.3s
[CV] END ......learning_rate=0.01, max_depth=9, max_iter=300; total time=   4.3s
[CV] END ......learning_rate=0.01, max_depth=9, max_iter=300; total time=   4.3s
[CV] END ......learning_rate=0.01, max_depth=9, max_iter=300; total time=   4.3s
[CV] END ......learning_rate=0.05, max_depth=6, max_iter=200; total time=   1.7s
[CV] END ......learning_rate=0.05, max_depth=6, max_iter=200; total time=   1.9s
[CV] END ......learning_rate=0.05, max_depth=6, max_iter=200; total time=   1.6s
[CV] END ......learning_rate=0.05, max_depth=6, max_iter=200; total time=   2.2s
[CV] END ......learning_rate=0.05, max_depth=6, max_iter=200; total time=   2.4s
[CV] END ......learning_rate=0.05, max_depth=3, max_iter=100; total time=   0.8s
[CV] END ......learning_rate=0.05, max_depth=3, max_iter=100; total time=   0.8s
[CV] END ......learning_rate

In [20]:
import numpy as np
import pandas as pd

# Load the previously tuned models and their predictions
submission_cat = pd.read_csv('submission_cat.csv')
submission_rf = pd.read_csv('submission_rf.csv')
submission_xgb = pd.read_csv('submission_xgb.csv')
submission_gb = pd.read_csv('submission_gb.csv')

# Combine the predictions from different models
ensemble_pred = (submission_cat['Age'] + submission_rf['Age'] + submission_xgb['Age'] + submission_gb['Age']) / 4

# Round the ensemble predictions
ensemble_pred = np.round(ensemble_pred)

# Update the submission DataFrame with ensemble predictions
submission["Age"] = ensemble_pred

# Save ensemble predictions to a CSV file
submission.to_csv('submission_ensemble.csv', index=False)

# Read the ensemble submission file
submission_ensemble = pd.read_csv('submission_ensemble.csv')
print(submission_ensemble)


           id   Age
0       74051   8.0
1       74052   8.0
2       74053  10.0
3       74054   9.0
4       74055   7.0
...       ...   ...
49363  123414   9.0
49364  123415   8.0
49365  123416  14.0
49366  123417  10.0
49367  123418  12.0

[49368 rows x 2 columns]
